# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# load CSV into DataFrame
city_df=pd.read_csv("cities.csv")
city_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,port elizabeth,75,ZA,1603646148,77,-33.92,25.57,66.20,12.75
1,1,busselton,0,AU,1603646148,58,-33.65,115.33,57.99,2.75
2,2,mahebourg,75,MU,1603645859,73,-20.41,57.70,75.20,14.99
3,3,victoria,95,HK,1603646104,79,22.29,114.16,75.00,1.01
4,5,rikitea,100,PF,1603646149,88,-23.12,-134.97,71.38,17.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#from api_keys import g_key
gmaps.configure(api_key=g_key) 
locations = city_df[['Lat', 'Lng']]
weights = city_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# create new DataFrame with my conditions: Wind Speed >10, Max Temp >60 and <70 and Cloudiness<3
weather_city_df=city_df[(city_df['Wind Speed']>10)&(city_df['Max Temp']>60)&(city_df['Max Temp']<70)&(city_df['Cloudiness']<3)]
weather_city_df                                                                                                                              

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,40,east london,0,ZA,1603646153,77,-33.02,27.91,64.40,13.87
40,45,constitucion,0,CL,1603646153,52,-35.33,-72.42,60.12,15.17
225,238,asfi,0,MA,1603646139,67,32.30,-9.24,67.53,18.12
299,319,le pradet,0,FR,1603646188,82,43.11,6.02,66.20,11.41
372,401,pisco,0,PE,1603646198,72,-13.70,-76.22,68.00,10.29
517,562,rawson,0,AR,1603646219,24,-43.30,-65.10,67.39,21.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# dropped NA's and reset index
hotel_df=weather_city_df.dropna().reset_index(drop=True)
hotel_df.insert(1,'Hotel Name',None)
for index in range(0,hotel_df['City'].count()):
    city=hotel_df.iloc[index]['City']
    lat=hotel_df.iloc[index]['Lat']
    lng=hotel_df.iloc[index]['Lng']
    url=f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=hotel&key={g_key}'
    response=requests.get(url)
    hotel=response.json().get('results')[0].get('name')
    hotel_df.at[index,'Hotel Name']=hotel
    
hotel_df

,City_ID,Hotel Name,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,40,East London,east london,0,ZA,1603646153,77,-33.02,27.91,64.40,13.87
1,45,Constitución,constitucion,0,CL,1603646153,52,-35.33,-72.42,60.12,15.17
2,238,Safi,asfi,0,MA,1603646139,67,32.30,-9.24,67.53,18.12
3,319,Toulon,le pradet,0,FR,1603646188,82,43.11,6.02,66.20,11.41
4,401,Pisco,pisco,0,PE,1603646198,72,-13.70,-76.22,68.00,10.29
5,562,Rawson,rawson,0,AR,1603646219,24,-43.30,-65.10,67.39,21.34


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))